# Comprehensive Global Sensitivity Analysis Tutorial

This tutorial provides an in-depth exploration of GSA capabilities in MCPost, covering advanced techniques and interpretation strategies.

## Table of Contents

1. [Understanding GSA Metrics](#understanding-gsa-metrics)
2. [The Ishigami Function](#the-ishigami-function)
3. [Advanced GSA Configuration](#advanced-gsa-configuration)
4. [Interpreting Results](#interpreting-results)
5. [Handling Real-World Challenges](#handling-real-world-challenges)

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mcpost import gsa_pipeline, gsa_for_target
from mcpost import plot_sensitivity_metrics

# Set random seed for reproducibility
np.random.seed(42)

print("MCPost comprehensive GSA tutorial loaded!")

## Understanding GSA Metrics

MCPost provides multiple sensitivity metrics, each capturing different aspects of parameter influence:

### Mutual Information (MI)
- Measures statistical dependence between parameters and outputs
- Captures both linear and nonlinear relationships
- Range: [0, ∞), higher values indicate stronger influence
- Model-agnostic and distribution-free

### Distance Correlation (dCor)
- Measures dependence between random vectors
- Range: [0, 1], where 0 = independent, 1 = perfectly dependent
- Captures nonlinear and non-monotonic relationships
- Particularly useful for complex, multimodal relationships

### Permutation Importance
- Model-agnostic importance measure
- Computed by shuffling parameter values and measuring performance decrease
- Directly interpretable: how much does randomizing this parameter hurt predictions?
- Robust to model assumptions

### Sobol' Indices
- **Si (First-order)**: Main effect of parameter i
- **STi (Total)**: Total effect including all interactions involving parameter i
- **Interaction effects**: STi - Si measures interaction contributions
- Variance-based decomposition: Si + interactions = 1

### Gaussian Process ARD
- Automatic Relevance Determination from GP surrogate
- 1/ARD_LS indicates relative parameter importance
- Based on learned length scales in GP kernel
- Reflects how "relevant" each parameter is for GP predictions

## The Ishigami Function

The Ishigami function is a standard benchmark for GSA methods:

```
f(x1, x2, x3) = sin(x1) + 7*sin(x2)^2 + 0.1*x3^4*sin(x1)
```

**Theoretical Sobol' indices:**
- S1 = 0.314, S2 = 0.442, S3 = 0.000
- ST1 = 0.558, ST2 = 0.442, ST3 = 0.244

**Key insights:**
- x2 has the largest main effect (S2 = 0.442)
- x1 has moderate main effect but strong interactions (ST1 - S1 = 0.244)
- x3 has no main effect but contributes through interactions with x1

In [ ]:
# Define the Ishigami function
def ishigami_function(X):
    """
    Ishigami function: f(x1,x2,x3) = sin(x1) + 7*sin(x2)^2 + 0.1*x3^4*sin(x1)
    
    Parameters should be in range [-π, π]
    """
    x1, x2, x3 = X[:, 0], X[:, 1], X[:, 2]
    return np.sin(x1) + 7 * np.sin(x2)**2 + 0.1 * x3**4 * np.sin(x1)

# Generate samples
n_samples = 5000
X_ishigami = np.random.uniform(-np.pi, np.pi, (n_samples, 3))

# Evaluate function
y_ishigami = ishigami_function(X_ishigami)
Y_ishigami = y_ishigami.reshape(-1, 1)

print(f"Generated {n_samples} Ishigami function samples")
print(f"Output range: [{y_ishigami.min():.3f}, {y_ishigami.max():.3f}]")
print(f"Output std: {y_ishigami.std():.3f}")

In [ ]:
# Run comprehensive GSA on Ishigami function
param_names = ["x1", "x2", "x3"]
feature_names = ["ishigami"]

print("Running comprehensive GSA on Ishigami function...")
results_ishigami = gsa_pipeline(
    X_ishigami, Y_ishigami,
    param_names=param_names,
    feature_names=feature_names,
    scaler="minmax",
    enable_sobol=True,
    enable_gp=True,
    enable_perm=True,
    make_pdp=True,
    topk_pdp=3,
    N_sobol=8192  # Higher for better Sobol accuracy
)

# Display results
ishigami_table = results_ishigami["results"]["ishigami"]["table"]
print("\nIshigami Function - GSA Results:")
print(ishigami_table)

# Compare with theoretical values
theoretical = {"x1": {"Si": 0.314, "STi": 0.558}, 
               "x2": {"Si": 0.442, "STi": 0.442}, 
               "x3": {"Si": 0.000, "STi": 0.244}}

print("\nComparison with theoretical Sobol indices:")
print(f"{'Param':<5} {'Si_computed':<12} {'Si_theory':<11} {'STi_computed':<13} {'STi_theory':<12}")
print("-" * 65)
for param in param_names:
    si_comp = ishigami_table.loc[param, 'Si'] if 'Si' in ishigami_table.columns else 0
    sti_comp = ishigami_table.loc[param, 'STi'] if 'STi' in ishigami_table.columns else 0
    si_theory = theoretical[param]['Si']
    sti_theory = theoretical[param]['STi']
    print(f"{param:<5} {si_comp:<12.3f} {si_theory:<11.3f} {sti_comp:<13.3f} {sti_theory:<12.3f}")

In [ ]:
# Visualize sensitivity results
fig, ax = plot_sensitivity_metrics(
    ishigami_table,
    title="Ishigami Function - Comprehensive Sensitivity Analysis",
    figsize=(12, 8)
)
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- x2 shows highest sensitivity across all metrics (expected)")
print("- x1 shows moderate main effect but high total effect (interactions with x3)")
print("- x3 shows low main effect but contributes through interactions")
print("- Different metrics provide complementary insights")

## Advanced GSA Configuration

MCPost provides many configuration options to customize your analysis:

In [ ]:
# Compare different scaling options
scaling_options = ['minmax', 'standard', None]
scaling_results = {}

print("Comparing scaling options on Ishigami function:")
print("=" * 50)

for scaler in scaling_options:
    print(f"\nTesting scaler: {scaler}")
    
    result = gsa_for_target(
        X_ishigami, y_ishigami,
        param_names=param_names,
        scaler=scaler,
        enable_perm=True,
        enable_gp=True,
        enable_sobol=True,
        N_sobol=4096
    )
    
    scaling_results[str(scaler)] = result
    
    # Show key metrics
    print(f"  MI ranking: {result.sort_values('MI', ascending=False).index.tolist()}")
    if 'Si' in result.columns:
        print(f"  Sobol Si ranking: {result.sort_values('Si', ascending=False).index.tolist()}")

print("\nNote: Different scalers can affect GP-based metrics but shouldn't change rankings significantly")

In [ ]:
# Compare different kernel types for GP
kernel_options = ['rbf', 'matern32', 'matern52']
kernel_results = {}

print("Comparing GP kernel options:")
print("=" * 30)

for kernel in kernel_options:
    print(f"\nTesting kernel: {kernel}")
    
    result = gsa_for_target(
        X_ishigami, y_ishigami,
        param_names=param_names,
        scaler='minmax',
        kernel_kind=kernel,
        enable_gp=True,
        enable_perm=False,  # Skip for speed
        enable_sobol=False  # Skip for speed
    )
    
    kernel_results[kernel] = result
    
    # Show GP-based metrics
    if '1/ARD_LS' in result.columns:
        ard_ranking = result.sort_values('1/ARD_LS', ascending=False).index.tolist()
        print(f"  ARD ranking: {ard_ranking}")
        print(f"  ARD values: {result['1/ARD_LS'].round(3).to_dict()}")

print("\nNote: Different kernels encode different smoothness assumptions")
print("- RBF: Infinitely smooth (most common)")
print("- Matérn 3/2: Less smooth, more flexible")
print("- Matérn 5/2: Moderately smooth")

## Interpreting Results

Understanding what different metrics tell you about your model:

In [ ]:
# Create a function with known characteristics for interpretation
def interpretation_function(X):
    """
    Function designed to demonstrate different sensitivity patterns:
    - x1: Linear effect
    - x2: Quadratic effect  
    - x3: Interaction with x1
    - x4: Threshold effect
    - x5: Noise parameter (minimal effect)
    """
    x1, x2, x3, x4, x5 = X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4]
    
    linear_term = 2 * x1
    quadratic_term = x2**2
    interaction_term = x1 * x3
    threshold_term = np.where(x4 > 0, x4**2, 0)
    noise_term = 0.01 * x5
    
    return linear_term + quadratic_term + interaction_term + threshold_term + noise_term

# Generate samples
n_samples = 3000
X_interp = np.random.uniform(-1, 1, (n_samples, 5))
y_interp = interpretation_function(X_interp)

param_names_interp = ["x1_linear", "x2_quadratic", "x3_interaction", "x4_threshold", "x5_noise"]

print("Interpretation Function Analysis:")
print(f"Generated {n_samples} samples")
print(f"Expected ranking: x2 > x1 ≈ x3 > x4 > x5")

In [ ]:
# Run GSA on interpretation function
results_interp = gsa_for_target(
    X_interp, y_interp,
    param_names=param_names_interp,
    scaler='minmax',
    enable_perm=True,
    enable_gp=True,
    enable_sobol=True,
    N_sobol=4096
)

print("\nInterpretation Function - GSA Results:")
print(results_interp[['MI', 'dCor', 'PermMean', 'Si', 'STi']].round(4))

# Analyze rankings
metrics = ['MI', 'dCor', 'PermMean', 'Si', 'STi']
print("\nRankings by different metrics:")
for metric in metrics:
    if metric in results_interp.columns:
        ranking = results_interp.sort_values(metric, ascending=False).index.tolist()
        print(f"{metric:<12}: {ranking}")

print("\nInterpretation insights:")
print("- x2_quadratic should rank high (strong nonlinear effect)")
print("- x1_linear should rank high (strong linear effect)")
print("- x3_interaction shows up in total effects (STi) more than main effects (Si)")
print("- x4_threshold may show different rankings depending on metric sensitivity to discontinuities")
print("- x5_noise should consistently rank lowest")

## Handling Real-World Challenges

Real-world GSA often involves challenges like correlated inputs, missing data, and computational constraints:

In [ ]:
# Challenge 1: Correlated inputs
print("Challenge 1: Handling Correlated Inputs")
print("=" * 40)

# Generate correlated inputs
n_samples = 2000
mean = [0, 0, 0]
cov = [[1.0, 0.7, 0.3],    # x1 strongly correlated with x2
       [0.7, 1.0, 0.1],    # x2 moderately correlated with x3
       [0.3, 0.1, 1.0]]    # x3 weakly correlated with others

X_corr = np.random.multivariate_normal(mean, cov, n_samples)

# Simple linear model
def correlated_model(X):
    return 2*X[:, 0] + X[:, 1] + 0.5*X[:, 2]

y_corr = correlated_model(X_corr)

# Run GSA
results_corr = gsa_for_target(
    X_corr, y_corr,
    param_names=["x1", "x2", "x3"],
    scaler='standard',  # Better for correlated normal variables
    enable_perm=True,
    enable_sobol=True
)

print("\nCorrelated inputs - GSA results:")
print(results_corr[['MI', 'dCor', 'PermMean', 'Si']].round(4))

# Check correlation matrix
corr_matrix = np.corrcoef(X_corr.T)
print("\nInput correlation matrix:")
print(pd.DataFrame(corr_matrix, index=["x1", "x2", "x3"], columns=["x1", "x2", "x3"]).round(3))

print("\nNote: With correlated inputs, interpretation becomes more complex.")
print("Permutation importance may be more reliable than Sobol indices.")

In [ ]:
# Challenge 2: Computational efficiency for large problems
print("\nChallenge 2: Computational Efficiency")
print("=" * 35)

# Simulate a larger problem
n_params_large = 20
n_samples_large = 5000

print(f"Large problem: {n_params_large} parameters, {n_samples_large} samples")

# Generate data
X_large = np.random.uniform(-1, 1, (n_samples_large, n_params_large))

# Simple additive model with decreasing importance
def large_model(X):
    weights = np.exp(-np.arange(X.shape[1]) * 0.3)  # Exponentially decreasing importance
    return np.sum(X * weights, axis=1)

y_large = large_model(X_large)
param_names_large = [f"param_{i+1}" for i in range(n_params_large)]

# Fast GSA configuration (skip expensive methods)
import time
start_time = time.time()

results_large = gsa_for_target(
    X_large, y_large,
    param_names=param_names_large,
    scaler='minmax',
    enable_perm=False,    # Skip permutation importance (expensive)
    enable_gp=True,       # Keep GP (relatively fast)
    enable_sobol=False,   # Skip Sobol (expensive for many parameters)
)

elapsed_time = time.time() - start_time

print(f"\nFast GSA completed in {elapsed_time:.2f} seconds")
print("\nTop 10 most important parameters:")
top_params = results_large.sort_values('MI', ascending=False).head(10)
print(top_params[['MI', 'dCor']].round(4))

print("\nComputational tips:")
print("- Use MI and dCor for quick screening")
print("- Enable GP for moderate-sized problems (< 50 parameters)")
print("- Use Sobol indices only for final analysis of top parameters")
print("- Consider chunked processing for very large datasets")

## Summary and Best Practices

This comprehensive tutorial covered:

1. **Multiple GSA metrics** and their interpretations
2. **Benchmark testing** with the Ishigami function
3. **Configuration options** for different scenarios
4. **Result interpretation** strategies
5. **Real-world challenges** and solutions

### Best Practices:

1. **Start with fast methods** (MI, dCor) for initial screening
2. **Use multiple metrics** for robust conclusions
3. **Consider input correlations** when interpreting results
4. **Validate with known benchmarks** when possible
5. **Choose appropriate scaling** based on your parameter distributions
6. **Use sufficient sample sizes** (> 1000 for reliable results)

### When to use each metric:

- **MI & dCor**: General-purpose, fast, good for screening
- **Permutation Importance**: When you have a good predictive model
- **Sobol Indices**: For variance decomposition and interaction analysis
- **GP ARD**: When you want to fit a surrogate model anyway

### Next Steps:

- Apply these techniques to your own models
- Experiment with different configurations
- Combine GSA with Monte Carlo integration for comprehensive analysis
- Explore the MCPost extension system for custom methods